# Logistic Regression Model

In [15]:
import pandas as pd

#df gives the original dataset
df = pd.read_csv('./train_dataset.csv.xls')
#df_test = pd.read_csv('./loan_test.csv')

# drop unnecessary columns
#cols = 
df=df.drop(['Loan_ID', 'CoapplicantIncome'], 1)
df.head(5)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849,NaN,360.0,1.0,Urban,Y
1,Male,Yes,1,Graduate,No,4583,128.0,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,66.0,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,120.0,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,141.0,360.0,1.0,Urban,Y


In [16]:
#process data before feeding it into the model
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

#encode categorical attributes
encode=['Gender', 'Married', 'Education', 'Self_Employed', 'Property_Area', 'Loan_Status', 'Dependents']
encoder=LabelEncoder()
for col in encode:
    df[col]=encoder.fit_transform(df[col])
    
#split data into train and test
train, test = train_test_split(df, test_size=0.2, random_state=21)

#extract dependent and indepedent vars
X_train, y_train = train.drop(columns=['Loan_Status']), train['Loan_Status']
X_test, y_test = test.drop(columns=['Loan_Status']), test['Loan_Status']


#scale the data using Standard Scaler
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [19]:
#model using logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
import numpy as np

lr_model = LogisticRegression()
lr_model.fit(X_train,y_train)
print("Accuracy is ", lr_model(X_test,y_test)*100)
cv_score = cross_val_score(lr_model.copy().drop(columns=['Loan_Status']),df['Loan_Status'],cv=5)
print("Cross validation is ," ,np.mean(score)*100)
y_pred=lr_model.predict(x_test)
print(confusionmatrix(y_pred,y_test))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').